In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_probability as tfp
from tensorflow.keras.layers import Dense, LeakyReLU, LSTM
import import_ipynb
import Experience_Replay
from tensorflow.keras.optimizers import Adam

importing Jupyter notebook from Experience_Replay.ipynb


In [2]:
#physical_device = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_device[0], True)

In [2]:
initial_learning_rate = 3e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)

In [3]:
class CriticNetwork(keras.Model):
    def __init__(self, n_actions, fc1_dims=256, fc2_dims=256,  name='criticB', chkpt_dir='tmpB/sacB'):
        super(CriticNetwork, self).__init__()
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.model_name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name+'_sacB')
        
        self.fc1 = Dense(self.fc1_dims, activation=LeakyReLU())
        self.fc2 = Dense(self.fc2_dims, activation=LeakyReLU())
        self.q = Dense(1, activation=None)
    def call(self, state, action):
        action_value = self.fc1(tf.concat([state, action], axis=1))
        action_value = self.fc2(action_value)
      
        
        q = self.q(action_value)
        
        return q
    

class ValueNetwork(keras.Model):
    def __init__(self, n_actions, fc1_dims=256, fc2_dims=256,  name='valueB', chkpt_dir='tmpB/sacB'):
        super(ValueNetwork, self).__init__()
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.model_name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name+'_sacB')
        
        self.fc1 = Dense(self.fc1_dims, activation=LeakyReLU())
        self.fc2 = Dense(self.fc2_dims, activation=LeakyReLU())

        self.v = Dense(1, activation=None)
    def call(self, state):
        state_value = self.fc1(state)
        state_value = self.fc2(state_value)
     
        
        v = self.v(state_value)
        
        return v

class ActorNetwork(keras.Model):
    def __init__(self, max_action, fc1_dims=256, fc2_dims=256, n_actions = 1,  name='actorB', chkpt_dir='tmpB/sacB'):
        super(ActorNetwork, self).__init__()
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.max_action = max_action
        self.model_name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name+'_sacB')
        self.noise = 1e-6 #1e-2 or 0.01
        
        self.fc1 = Dense(self.fc1_dims, activation=LeakyReLU())
        self.fc2 = Dense(self.fc2_dims, activation=LeakyReLU())
      
        
        self.mu =  Dense(self.n_actions, activation=None)
     
        self.sigma = Dense(self.n_actions, activation=None)
    def call(self, state):
        prob = self.fc1(state)
 
        prob = self.fc2(prob)


        
        
        mu = self.mu(prob)
        sigma = self.sigma(prob)

        sigma = tf.clip_by_value(sigma, self.noise, 1)
        
        return mu, sigma
    def sample_normal(self, state):
        mu, sigma = self.call(state)
        #sigma = 1e-6
        
        probabilities = tfp.distributions.Normal(mu, sigma)
    
        actions =  probabilities.sample()
      
     
        action = tf.math.tanh(actions)*self.max_action
  
        
    
        
        log_probs = probabilities.log_prob(actions)
     
        
        log_probs = log_probs - tf.math.log(1-tf.math.pow(action, 2)+self.noise)

        log_probs = tf.math.reduce_sum(log_probs, axis=1, keepdims=True)
      
        
        return action, log_probs
        

In [4]:
class AgentB:
    def __init__(self, alpha=lr_schedule, beta=lr_schedule, input_dims=[12], env=None,
                gamma=0.99, n_actions=3, max_size=1000000, tau=0.005, layer1_size=256 
                , layer2_size = 256, batch_size=256, reward_scale =1):
        self.gamma = gamma
        self.tau = tau
        self.memory = Experience_Replay.EP(max_size)
        self.batch_size = batch_size
        self.n_actions = n_actions
        
        self.actor= ActorNetwork(n_actions=n_actions,
                                max_action=1)
        self.critic_1 = CriticNetwork(n_actions=n_actions)
        self.critic_2 = CriticNetwork(n_actions=n_actions)
        self.value = ValueNetwork(n_actions=n_actions)
        self.target_value = ValueNetwork(n_actions=n_actions)
        
        self.actor.compile(optimizer=Adam(learning_rate=alpha))
        self.critic_1.compile(optimizer=Adam(learning_rate= beta))
        self.critic_2.compile(optimizer=Adam(learning_rate= beta))
        self.value.compile(optimizer=Adam(learning_rate= beta))
        self.target_value.compile(optimizer=Adam(learning_rate= beta))
        
        self.scale = reward_scale
        self.update_network_parameters(tau=1)
    def choose_action(self, observation, epi_time, start):
        if start == True:
            s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18 = observation[0]
            
           
            observation = np.array([s1[0], s2[0],s3[0]/1000,s4[0],s5[0],s6[0]/1000,s7,s8,s9,s10[0]/1000,s11[0],s12[0],s13[0],s14[0],s15[0],s16[0],s17[0],s18[0]])
            
        else:
            s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18 = observation[0]
            
          
            observation = np.array([s1[epi_time], s2[epi_time],s3[epi_time]/1000,s4[epi_time],s5[epi_time],s6[epi_time]/1000,s7,s8,s9,s10[epi_time]/1000,s11[epi_time],s12[epi_time],s13[epi_time],s14[epi_time],s15[epi_time],s16[epi_time],s17[epi_time],s18[epi_time]])
                       
        state = tf.convert_to_tensor([observation])
        
        actions,l = self.actor.sample_normal(state)
        return actions[0]
    def remember(self, state, action, reward, new_state, done):
        #self.memory.update(state, action, reward, new_state, done)
        self.exp_1 = self.memory.update(state, action, reward, new_state, done)
        self.done = done
    def update_network_parameters(self, tau=None):
        if tau is None:
            tau= self.tau
        weights = []
        targets = self.target_value.weights
        for i, weight in enumerate(self.target_value.weights):
            weights.append(weight*tau + targets[i]*(1-tau))
        self.target_value.set_weights(weights)
        
    def save_models(self):
        print('... saving models ...')
        self.actor.save_weights(self.actor.checkpoint_file)
        self.critic_1.save_weights(self.critic_1.checkpoint_file)
        self.critic_2.save_weights(self.critic_2.checkpoint_file)
        self.value.save_weights(self.value.checkpoint_file)
        self.target_value.save_weights(self.target_value.checkpoint_file)

    def load_models(self):
        print('... loading models ...')
        self.actor.load_weights(self.actor.checkpoint_file)
        self.critic_1.load_weights(self.critic_1.checkpoint_file)
        self.critic_2.load_weights(self.critic_2.checkpoint_file)
        self.value.load_weights(self.value.checkpoint_file)
        self.target_value.load_weights(self.target_value.checkpoint_file)

    
    def learn(self,aplh):
        exp_length = len(self.exp_1)
        if exp_length < 10000: 
            return
        #print('Learning.......')
        self.batch_train = self.memory.sample(self.batch_size)
        state = []
        action = []
        reward = []
        new_state = []
        done = []
        for i in range(len(self.batch_train)):
            state.append(self.batch_train[i][0])
            action.append(self.batch_train[i][1])
            reward.append(self.batch_train[i][2])
            new_state.append(self.batch_train[i][3])
            done.append(self.batch_train[i][4])
        
        
        state =np.asarray(state, dtype=np.float32)
        action =np.asarray(action, dtype=np.float32)
        reward = np.asarray(reward, dtype=np.float32)
        new_state =np.asarray(new_state, dtype=np.float32)
        done =np.asarray(done, dtype=np.float32)
        
        
        
            
        states = tf.convert_to_tensor(state, dtype=tf.float32)
        states_ = tf.convert_to_tensor(new_state, dtype=tf.float32)
        rewards = tf.convert_to_tensor(reward, dtype=tf.float32)
        actions = tf.convert_to_tensor(action, dtype=tf.float32)
        
        with tf.GradientTape() as tape:
            value = tf.squeeze(self.value(states), 1)
           
            value_ = tf.squeeze(self.target_value(states_), 1)
            
            current_policy_actions, log_probs = self.actor.sample_normal(states)
            log_probs = tf.squeeze(log_probs, 1)
            q1_new_policy = self.critic_1(states, current_policy_actions)
            q2_new_policy = self.critic_1(states, current_policy_actions)
            critic_value = tf.squeeze(tf.math.minimum(q1_new_policy, q2_new_policy), 1)
            value_target =critic_value-(aplh*log_probs)#log_probs(aplh*-2)
            value_loss = 0.5* keras.losses.MSE(value, value_target)
           
        value_network_gradient = tape.gradient(value_loss, self.value.trainable_variables)
            
        self.value.optimizer.apply_gradients(zip(value_network_gradient, self.value.trainable_variables ))
    
        with tf.GradientTape() as tape:

            new_policy_actions, log_probs = self.actor.sample_normal(states)
            log_probs = tf.squeeze(log_probs, 1)
            q1_new_policy = self.critic_1(states, new_policy_actions)
            q2_new_policy = self.critic_1(states, new_policy_actions)  
            
            critic_value = tf.squeeze(tf.math.minimum(q1_new_policy, q2_new_policy), 1)
            
            actor_loss = (aplh*log_probs) -critic_value #log_probs(aplh*-2)
            #print(tf.math.reduce_mean(log_probs/critic_value))
            actor_loss = tf.math.reduce_mean(actor_loss)
            print(tf.math.reduce_mean(log_probs))
            
        actor_network_gradient =  tape.gradient(actor_loss, self.actor.trainable_variables)
        
        self.actor.optimizer.apply_gradients(zip(actor_network_gradient, self.actor.trainable_variables ))
        
        with tf.GradientTape(persistent=True) as tape:
            
            q_hat = self.scale*rewards + self.gamma*value_*(1-int(0)) #entropy term
            print(tf.math.reduce_mean(rewards))
            print(tf.math.reduce_mean(value_))
            q1_old_policy = tf.squeeze(self.critic_1(state, action), 1)
            q2_old_policy = tf.squeeze(self.critic_2(state, action), 1)
           
            critic_1_loss = 0.5*keras.losses.MSE(q1_old_policy, q_hat)
            critic_2_loss =  0.5*keras.losses.MSE(q2_old_policy, q_hat)
          
        critic_1_network_gradient = tape.gradient(critic_1_loss, self.critic_1.trainable_variables)
        critic_2_network_gradient = tape.gradient(critic_2_loss, self.critic_2.trainable_variables)
            
        self.critic_1.optimizer.apply_gradients(zip(critic_1_network_gradient, self.critic_1.trainable_variables))
        self.critic_2.optimizer.apply_gradients(zip(critic_2_network_gradient, self.critic_2.trainable_variables))
        
        self.update_network_parameters()
                
            
            